In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [2]:
data_pollution = pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert", encoding = 'utf-8')

In [3]:
base_geo_pour_merge = pd.read_csv("https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv", encoding = 'utf-8')

In [4]:
base_gen_pollution = pd.merge(data_pollution, base_geo_pour_merge, how='inner', 
                              left_on='INSEE commune', right_on='COM')

In [5]:
merging_list = data_pollution.columns

In [6]:
merging_list = list(merging_list)

In [7]:
merging_list.append('REG')

In [8]:
merging_list.append('DEP')

In [9]:
print(merging_list)

['INSEE commune', 'Commune', 'Agriculture', 'Autres transports', 'Autres transports international', 'CO2 biomasse hors-total', 'Déchets', 'Energie', 'Industrie hors-énergie', 'Résidentiel', 'Routier', 'Tertiaire', 'REG', 'DEP']


In [10]:
base_gen_pollution = base_gen_pollution[merging_list]

In [11]:
base_gen_pollution.head()

,INSEE commune,Commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire,REG,DEP
0,01001,L'ABERGEMENT-CLEMENCIAT,3711.425991,NaN,NaN,432.751835,101.430476,2.354558,6.911213,309.358195,793.156501,367.036172,84.0,01
1,01002,L'ABERGEMENT-DE-VAREY,475.330205,NaN,NaN,140.741660,140.675439,2.354558,6.911213,104.866444,348.997893,112.934207,84.0,01
2,01004,AMBERIEU-EN-BUGEY,499.043526,212.577908,NaN,10313.446515,5314.314445,998.332482,2930.354461,16616.822534,15642.420313,10732.376934,84.0,01
3,01005,AMBERIEUX-EN-DOMBES,1859.160954,NaN,NaN,1144.429311,216.217508,94.182310,276.448534,663.683146,1756.341319,782.404357,84.0,01
4,01006,AMBLEON,448.966808,NaN,NaN,77.033834,48.401549,NaN,NaN,43.714019,398.786800,51.681756,84.0,01


In [12]:
base_gen_pollution['REG'].unique()

array([84., nan, 32., 93., 44., 76., 28., 75., 24., 27., 53., 94., 52.,
       11.])

In [13]:
base_gen_pollution.shape

(36196, 14)

In [14]:
len(base_gen_pollution['REG'].unique())

14

In [15]:
len(base_gen_pollution['DEP'].unique())

97

In [16]:
#probleme pour plus tard
#differences avec le nombre de regions et de departement dans l autre table 

In [17]:
tentative_groupes_1 = base_gen_pollution.drop(['Commune'], axis = 1).groupby('DEP').sum()

In [18]:
tentative_groupes_1.head()

,INSEE commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire,REG
DEP,,,,,,,,,,,,
01,0100101002010040100501006010070100801009010100...,7.816819e+05,15372.396872,258.110863,703148.137380,270526.994907,107108.831646,704385.028742,546437.687545,1.599545e+06,412255.462975,32928.0
02,0200102002020030200402005020060200702008020090...,1.270742e+06,57904.229456,2476.899590,615899.788193,180007.937389,60344.960348,737257.067508,637882.728847,1.381081e+06,324154.207262,25536.0
03,0300103002030030300403005030060300703008030090...,1.947087e+06,19686.272920,545.159738,565740.821993,111205.867274,102975.024821,457513.404309,445664.842850,1.164099e+06,224438.681860,26628.0
04,0400104004040050400604007040080400904012040130...,3.604105e+05,5851.619941,0.000000,115775.438355,50497.301309,12184.836311,60899.229209,116640.241600,3.882993e+05,98069.382968,18414.0
05,0500105003050040500605007050080500905010050110...,3.003535e+05,6216.001726,0.000000,83727.598246,22099.068688,5453.155729,16006.370120,121515.999855,3.441621e+05,77274.057096,15066.0


In [19]:
tentative_groupes_1 = tentative_groupes_1.drop(['INSEE commune', 'REG'], axis = 1)

In [20]:
tentative_groupes_1.head()

,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire
DEP,,,,,,,,,,
01,7.816819e+05,15372.396872,258.110863,703148.137380,270526.994907,107108.831646,704385.028742,546437.687545,1.599545e+06,412255.462975
02,1.270742e+06,57904.229456,2476.899590,615899.788193,180007.937389,60344.960348,737257.067508,637882.728847,1.381081e+06,324154.207262
03,1.947087e+06,19686.272920,545.159738,565740.821993,111205.867274,102975.024821,457513.404309,445664.842850,1.164099e+06,224438.681860
04,3.604105e+05,5851.619941,0.000000,115775.438355,50497.301309,12184.836311,60899.229209,116640.241600,3.882993e+05,98069.382968
05,3.003535e+05,6216.001726,0.000000,83727.598246,22099.068688,5453.155729,16006.370120,121515.999855,3.441621e+05,77274.057096


In [21]:
tentative_groupes_1.shape

(96, 10)

In [22]:
set_dep_littoral = (59, 62, 80, 76, 27, 14, 50, 35, 
                      22, 29, 56, 44, 85, 17, 33, 40, 
                    64, 66, 11, 34,
                    30, 13, 83, 0o6, '2A', '2B')

In [23]:
print(set_dep_littoral)

(59, 62, 80, 76, 27, 14, 50, 35, 22, 29, 56, 44, 85, 17, 33, 40, 64, 66, 11, 34, 30, 13, 83, 6, '2A', '2B')


In [24]:
base_gen_pollution['DEP'].unique()

array(['01', nan, '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31',
       '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42',
       '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53',
       '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64',
       '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75',
       '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86',
       '87', '88', '89', '90', '91', '92', '93', '94', '95'], dtype=object)

In [25]:
#on a pas les DROM donc complique par rapport a l'autre base 

In [26]:
base_gen_pollution.to_csv('base_pollution.csv', sep=',', index=False, encoding='utf-8')
